# Problèmes

### 1)
Primal: $\underset{x \in \mathbb{R}^3}{\min} \frac{1}{2} x' Q x + c' x ~~~~~~~~ s.t. ~ A x = b ~, ~~ x \ge 0$

with  $Q = \begin{bmatrix} 6 & 2 & 1\\ 2 & 5 & 2\\ 1 & 2 & 4 \end{bmatrix} $, $c = \begin{bmatrix} -8 \\ -3 \\ -3 \end{bmatrix}$, $A = \begin{bmatrix} 1 & 0 &1 \\ 0 & 1 & 1 \end{bmatrix}$ , and $b = \begin{bmatrix} 3 \\ 0 \end{bmatrix} $.


### 2) 
AFIRO        28 rows    32 cols        88 nonzeros       794 bytes      -4.6475314286E+02 optimalvalue

In [1]:
using LinearAlgebra
using LaTeXStrings
using DataFrames
using DataStructures
using QuadraticModels
using Printf
using SparseArrays
using BenchmarkTools
using NLPModels
using LinearOperators
using QPSReader
using SolverTools
using SolverBenchmark
using LDLFactorizations
using Statistics
using Quadmath

# Mehrotra formulation plus générale:

min $c_0 + c'x + \frac{1}{2} x'Hx ~~~~$ s.t. $ Ax = b $ and $l \le x \le u$ 

In [2]:
function starting_points(Qrows, Qcols, Qvals, Arows, Acols, Avals, b, c, 
                         lvar, uvar, ilow, iupp, irng, J_augm, n_rows, n_cols, Δ_xλ)

    T = eltype(Avals)
    J_P = ldl_analyze(Symmetric(J_augm, :U))
    J_fact = ldl_factorize!(Symmetric(J_augm, :U), J_P)
    #J_fact = ldlt(Symmetric(J_augm-Diagonal(tmp_diag), :U))
#     J_fact = ldl(Symmetric(J_augm, :U))
#     J_P = J_fact.P
    Δ_xλ[n_cols+1: end] = b 
    Δ_xλ = ldiv!(J_fact, Δ_xλ)
    #init_xλ2 = J_fact \ [c ; zeros(n_rows)]
    x0 = Δ_xλ[1:n_cols]
    λ0 = Δ_xλ[n_cols+1:end]
    s0_l, s0_u = zeros(n_cols), zeros(n_cols)
    Qx, ATλ = zeros(n_cols), zeros(n_cols)
    Qx = mul_Qx_COO!(Qx, Qrows, Qcols, Qvals, x0)
    ATλ = mul_ATλ_COO!(ATλ, Arows, Acols, Avals, λ0)
    dual_val = Qx - ATλ + c
    s0_l[ilow] = @views dual_val[ilow]
    s0_u[iupp] = @views -dual_val[iupp]

    x0_m_lvar = @views x0[ilow] - lvar[ilow] 
    uvar_m_x0 = @views uvar[iupp] - x0[iupp]
    if length(ilow) == 0
        δx_l1, δs_l1 = zero(T), zero(T)
    else
        δx_l1 = max(-T(1.5)*minimum(x0_m_lvar), T(1.e0))
        δs_l1 = @views max(-T(1.5)*minimum(s0_l[ilow]), T(1.e-4))
    end
    
    if length(iupp) == 0
        δx_u1, δs_u1 = zero(T), zero(T)
    else
        δx_u1 = max(-T(1.5)*minimum(uvar_m_x0), T(1.e0))
        δs_u1 = @views max(-T(1.5)*minimum(s0_u[iupp]), T(1.e-4))
    end
    
    x0_m_lvar .+= δx_l1
    uvar_m_x0 .+= δx_u1
    s0_l1 = @views s0_l[ilow] .+ δs_l1
    s0_u1 = @views s0_u[iupp] .+ δs_u1
    xs_l1, xs_u1 = s0_l1' * x0_m_lvar, s0_u1' * uvar_m_x0
    if length(ilow) == 0
        δx_l2, δs_l2 = zero(T), zero(T)
    else
        δx_l2 = δx_l1 + xs_l1 / sum(s0_l1) / 2
        δs_l2 = @views δs_l1 + xs_l1 / sum(x0_m_lvar) / 2
    end
    if length(iupp) == 0
        δx_u2, δs_u2 = zero(T), zero(T)
    else
        δx_u2 = δx_u1 + xs_u1 / sum(s0_u1) / 2
        δs_u2 = @views δs_u1 + xs_u1 / sum(uvar_m_x0) / 2
    end
    δx = max(δx_l2, δx_u2)
    δs = max(δs_l2, δs_u2)
    x0[ilow] .+= δx
    x0[iupp] .-= δx
    s0_l[ilow] = @views s0_l[ilow] .+ δs
    s0_u[iupp] = @views s0_u[iupp] .+ δs

    @inbounds @simd for i in irng
#         if lvar[i] < x0[i] == false
#             x0[i] = lvar[i] + T(1e0)
#         elseif x0[i] < uvar[i] == false
#             x0[i] = uvar[i] - T(1e0)
#         end
        if (lvar[i] < x0[i] < uvar[i]) == false
            x0[i] = (lvar[i] + uvar[i]) / 2
        end
    end

    @assert all(x0 .> lvar) && all(x0 .< uvar)
    @assert @views all(s0_l[ilow] .> zero(T)) && all(s0_u[iupp] .> zero(T))

    return x0, λ0, s0_l, s0_u, J_P, Qx, ATλ, x0_m_lvar, uvar_m_x0, Δ_xλ
end



starting_points (generic function with 1 method)

In [3]:
function compute_α_dual(v, dir_v)
    n = length(v)
    T = eltype(v)
    if n == 0
        return one(T)
    end
    α = one(T)
    @inbounds @simd for i=1:n
        if dir_v[i] < zero(T)
            α_new = -v[i] * T(0.999) / dir_v[i]
            if α_new < α
                α = α_new
            end
        end
    end
    return α
end


    
function compute_α_primal(v, dir_v, lvar, uvar)
    n = length(v)
    T = eltype(v)
    α_l, α_u = one(T), one(T)
    @inbounds @simd for i=1:n
        if dir_v[i] > zero(T)
            α_u_new = (uvar[i] - v[i]) * T(0.999) / dir_v[i]
            if α_u_new < α_u
                α_u = α_u_new
            end
        elseif dir_v[i] < zero(T)
            α_l_new = (lvar[i] - v[i]) * T(0.999) / dir_v[i]
            if α_l_new < α_l
                α_l = α_l_new
            end
        end
    end
    return min(α_l, α_u)
end



function compute_μ(x_m_lvar, uvar_m_x, s_l, s_u, nb_low, nb_upp)
    return (s_l' * x_m_lvar + s_u' * uvar_m_x) / (nb_low + nb_upp)
end


function is_in_Neighborhood_inf(gamma, x_l, x_u, s_l, s_u, lvar, uvar)
    # check if the current point is in N_inf(gamma)
    # true : (xi_l - lvari) * si_l >= gamma mu   and   (uvari - xi_u) * si_u >= gamma mu 
    mu = Compute_mu(x_l, x_u, s_l, s_u, lvar, uvar)
    for i=1:length(x_l)
        if (x_l[i] - lvar[i]) * s_l[i] < gamma*mu
            return false
        end
    end
    for i=1:length(x_u)
        if (uvar[i] - x_u[i]) * s_u[i] < gamma*mu
            return false
        end
    end
    return true
end

is_in_Neighborhood_inf (generic function with 1 method)

In [4]:
function solve_augmented_system_aff!(J_fact, Δ_aff, Δ_xλ, rc, rb, x_m_lvar, uvar_m_x, 
                                     s_l, s_u, ilow, iupp,  n_cols, n_rows, n_low)

    Δ_xλ[1:n_cols] .= .-rc
    Δ_xλ[n_cols+1:end] .= .-rb
    Δ_xλ[ilow] += @views s_l[ilow]
    Δ_xλ[iupp] -= @views s_u[iupp]
    
    Δ_xλ = ldiv!(J_fact, Δ_xλ)

    Δ_aff[1:n_cols+n_rows] = Δ_xλ
    Δ_aff[n_cols+n_rows+1:n_cols+n_rows+n_low] = @views -s_l[ilow] - s_l[ilow].*Δ_xλ[1:n_cols][ilow]./x_m_lvar
    Δ_aff[n_cols+n_rows+n_low+1:end] = @views -s_u[iupp] + s_u[iupp].*Δ_xλ[1:n_cols][iupp]./uvar_m_x
    return Δ_aff
end

function solve_augmented_system_cc!(J_fact, Δ_cc, Δ_xλ ,Δ_aff, σ, μ, x_m_lvar, uvar_m_x, 
                                    rxs_l, rxs_u, s_l, s_u, ilow, iupp, n_cols, n_rows, n_low)


    rxs_l .= @views (-σ*μ .+ Δ_aff[1:n_cols][ilow].*Δ_aff[n_rows+n_cols+1: n_rows+n_cols+n_low])
    rxs_u .= @views σ*μ .+ Δ_aff[1:n_cols][iupp].*Δ_aff[n_rows+n_cols+n_low+1: end]

    Δ_xλ .= zero(eltype(Δ_xλ))
    Δ_xλ[ilow] .+= rxs_l./x_m_lvar
    Δ_xλ[iupp] .+= rxs_u./uvar_m_x

    Δ_xλ = ldiv!(J_fact, Δ_xλ)

    Δ_cc[1:n_cols+n_rows] = Δ_xλ
    Δ_cc[n_cols+n_rows+1:n_cols+n_rows+n_low] .= @views .-(rxs_l.+s_l[ilow].*Δ_xλ[1:n_cols][ilow])./x_m_lvar
    Δ_cc[n_cols+n_rows+n_low+1:end] .= @views (rxs_u.+s_u[iupp].*Δ_xλ[1:n_cols][iupp])./uvar_m_x 
    return Δ_cc
end

solve_augmented_system_cc! (generic function with 1 method)

In [5]:
function check_frontier!(γf, x, x_m_lvar, uvar_m_x, s_l, s_u, Δ, α_pri, α_dual, 
                        x_m_lvar_sup, uvar_m_x_sup, s_l_sup, s_u_sup,
                        lvar, uvar, ilow, iupp, ilow_pad, iupp_pad, 
                        n_low, n_upp, n_rows, n_cols)
    T = eltype(x)
    x_m_lvar_sup .= @views x_m_lvar .+ α_pri .* Δ[1:n_cols][ilow]
    uvar_m_x_sup .= @views uvar_m_x .- α_pri .* Δ[1:n_cols][iupp]
    s_l_sup .= @views s_l[ilow] .+ α_dual .* Δ[n_rows+n_cols+1: n_rows+n_cols+n_low]
    s_u_sup .= @views s_u[iupp] .+ α_dual .* Δ[n_rows+n_cols+n_low+1: end]
    μ_sup = compute_μ(x_m_lvar_sup, uvar_m_x_sup, s_l_sup, s_u_sup, n_low, n_upp)
    f_pri, f_pri_tmp = zero(T), zero(T)
    f_dual, f_dual_tmp = zero(T), zero(T)
    i_l, i_u = 0, 0

    @inbounds @simd for i=1:n_cols 
        if ilow_pad[i] != 0 && iupp_pad[i] != 0
            i_l, i_u = ilow_pad[i], iupp_pad[i]
            if (x_m_lvar_sup[i_l] == zero(T) || uvar_m_x_sup[i_u] == zero(T))
                f_pri_tmp = (2*γf*μ_sup - 
                            x_m_lvar[i_l]*(s_l_sup[i_l]+α_dual*Δ[n_rows+n_cols+i_l]) -
                            uvar_m_x[i_u]*(s_u_sup[i_u]+α_dual*Δ[n_rows+n_cols+n_low+i_u])) / 
                                (α_pri * Δ[i]*
                                (s_l_sup[i_l]+α_dual*Δ[n_rows+n_cols+i_l]-s_u_sup[i_u]-α_dual*Δ[n_rows+n_cols+n_low+i_u]))      
                if f_pri_tmp < f_pri
                    f_pri = f_pri_tmp
                end
            end
            if (s_l_sup[i_l] == zero(T) || uvar_m_x_sup[i_u] == zero(T))
                f_dual_tmp = (2*γf*μ_sup - s_l[i_l]*x_m_lvar_sup[i_l] - s_u[i_u]*uvar_m_x_sup[i_u]) /
                                (α_dual * (Δ[n_rows+n_cols+i_l]*x_m_lvar_sup[i_l] +
                                     Δ[n_rows+n_cols+n_low+i_u]*uvar_m_x_sup[i_u]))
                if f_dual_tmp < f_dual
                    f_dual = f_dual_tmp
                end  
            end
        elseif ilow_pad[i] != 0 && iupp_pad[i] == 0
            i_l = ilow_pad[i]
            if x_m_lvar_sup[i_l] == zero(T) 
                f_pri_tmp = (γf*μ_sup - x_m_lvar[i_l]*(s_l_sup[i_l]+α_dual*Δ[n_rows+n_cols+i_l])) / 
                                (α_pri * Δ[i]*(s_l_sup[i_l]+α_dual*Δ[n_rows+n_cols+i_l]))      
                if f_pri_tmp < f_pri
                    f_pri = f_pri_tmp
                end
            end
            if s_l_sup[i_l] == zero(T)
                f_dual_tmp = (γf*μ_sup - s_l[i_l]*x_m_lvar_sup[i_l] ) /
                                (α_dual * Δ[n_rows+n_cols+i_l]*x_m_lvar_sup[i_l])
                if f_dual_tmp < f_dual
                    f_dual = f_dual_tmp
                end
            end 
        elseif ilow_pad[i] == 0 && iupp_pad[i] != 0
            i_u = iupp_pad[i]
            if uvar_m_x_sup[i_u] == zero(T)
                f_pri_tmp = (γf*μ_sup - uvar_m_x[i_u]*(s_u_sup[i_u]+α_dual*Δ[n_rows+n_cols+n_low+i_u])) / 
                                (α_pri * Δ[i]*(-s_u_sup[i_u]-α_dual*Δ[n_rows+n_cols+n_low+i_u]))      
                if f_pri_tmp < f_pri
                    f_pri = f_pri_tmp
                end
            end
            if s_u[i_u] == zero(T)
                f_dual_tmp = (γf*μ_sup - s_u[i_u]*uvar_m_x_sup[i_u]) /
                            (α_dual * Δ[n_rows+n_cols+n_low+i_u]*uvar_m_x_sup[i_u])
                if f_dual_tmp < f_dual
                    f_dual = f_dual_tmp
                end  
            end
        end
    end
    
    α_pri *= max(one(T)-γf, f_pri)
    α_dual *= max(one(T)-γf, f_dual)
    
    return α_pri, α_dual
end  


check_frontier! (generic function with 1 method)

In [6]:
function mul_Qx_COO!(Qx, Qrows, Qcols, Qvals, x)
    # right mutiplication for sparse COO symetric matrix M: res=Mv
    Qx .= zero(eltype(Qx))
    @inbounds @simd for i=1:length(Qcols)
        Qx[Qrows[i]] += Qvals[i] * x[Qcols[i]] 
        if Qrows[i] != Qcols[i]
            Qx[Qcols[i]] += Qvals[i]*x[Qrows[i]]
        end
    end
    return Qx
end

function mul_ATλ_COO!(ATλ, Arows, Acols, Avals, λ)
    ATλ .= zero(eltype(ATλ))
    @inbounds @simd for i=1:length(Acols)
        ATλ[Acols[i]] += Avals[i] * λ[Arows[i]] 
    end
    return ATλ
end

function mul_Ax_COO!(Ax, Arows, Acols, Avals, x)
    Ax .= zero(eltype(Ax))
    @inbounds @simd for i=1:length(Acols)
        Ax[Arows[i]] += Avals[i] * x[Acols[i]] 
    end
    return Ax
end



function get_norm_rc!(v, A_i, Avals, n_v, n)
    T = eltype(v)
    v .= zero(T)
    @inbounds @simd for j=1:n
        if abs(Avals[j]) > v[A_i[j]] 
            v[A_i[j]] = abs(Avals[j])
        end
#         v[A_i[j]] += Avals[j]^2  #2-norm
    end
    
    v = sqrt.(v)
    @inbounds @simd for i=1:n_v
        if v[i] == zero(T)
            v[i] = one(T)
        end
    end
    return v
end

function mul_A_D1_D2!(Arows, Acols, Avals, d1, d2, r, c, n_rows, n_cols, n)
    @inbounds @simd for i=1:n
        Avals[i] /= r[Arows[i]] * c[Acols[i]]
    end
    d1 ./= r
    d2 ./= c
    return Arows, Acols, Avals, d1, d2
end
    
function mul_Q_D!(Qrows, Qcols, Qvals, d, c, n_cols, n)
    @inbounds @simd for i=1:n
        Qvals[i] /= c[Qrows[i]] * c[Qcols[i]]
    end
    d ./= c
    return Qrows, Qcols, Qvals, d
end

function scaling_Ruiz!(Arows, Acols, Avals, Qrows, Qcols, Qvals, c, b, lvar, uvar, 
                       n_rows, n_cols, ϵ; max_iter = 100)
    n = length(Arows)
    T = eltype(Avals)
    d1, d2 = ones(T, n_rows), ones(T, n_cols)
    r_k, c_k = zeros(T, n_rows), zeros(T, n_cols)
    
    r_k = get_norm_rc!(r_k, Arows, Avals, n_rows, n)
    c_k = get_norm_rc!(c_k, Acols, Avals, n_cols, n)
    convergence = maximum(abs.(one(T) .- r_k)) <= ϵ && maximum(abs.(one(T) .- c_k)) <= ϵ
    Arows, Acols, Avals, d1, d2 = mul_A_D1_D2!(Arows, Acols, Avals, d1, d2, 
                                               r_k, c_k, n_rows, n_cols, n)
    k = 1
    while !convergence && k < max_iter
        r_k = get_norm_rc!(r_k, Arows, Avals, n_rows, n)
        c_k = get_norm_rc!(c_k, Acols, Avals, n_cols, n)
        convergence = maximum(abs.(one(T) .- r_k)) <= ϵ && maximum(abs.(one(T) .- c_k)) <= ϵ
        Arows, Acols, Avals, d1, d2 = mul_A_D1_D2!(Arows, Acols, Avals, d1, d2, 
                                                   r_k, c_k, n_rows, n_cols, n)
        k += 1
    end
    
    n_Q = length(Qrows)
    @inbounds @simd for i=1:n_Q
        Qvals[i] *= d2[Qrows[i]] * d2[Qcols[i]]
    end
    b .*= d1
    c .*= d2
    lvar ./= d2
    uvar ./= d2
    
    # scaling Q (symmetric)
    d3 = ones(T, n_cols)
    c_k .= zero(T)
    c_k = get_norm_rc!(c_k, Qcols, Qvals, n_cols, n_Q)
    convergence = maximum(abs.(one(T) .- c_k)) <= ϵ
    Qrows, Qcols, Qvals, d3 = mul_Q_D!(Qrows, Qcols, Qvals, d3, c_k, n_cols, n_Q)
    k = 1
    while !convergence && k < max_iter
        c_k = get_norm_rc!(c_k, Qcols, Qvals, n_cols, n_Q)
        convergence = maximum(abs.(one(T) .- c_k)) <= ϵ
        Qrows, Qcols, Qvals, d3 = mul_Q_D!(Qrows, Qcols, Qvals, d3, c_k, n_cols, n_Q)
        k += 1
    end
    
    for i=1:n
        Avals[i] *= d3[Acols[i]]
    end
    c .*= d3
    lvar ./= d3
    uvar ./= d3

    return Arows, Acols, Avals, Qrows, Qcols, Qvals, c, b, lvar, uvar, d1, d2, d3
end


scaling_Ruiz! (generic function with 1 method)

In [7]:
function mul_A_D1!(Arows, Avals, d1, r, n_rows, n)
    @inbounds @simd for i=1:n
        Avals[i] /= r[Arows[i]]
    end
    d1 ./= r
    return Arows, Avals, d1
end


function scaling_Ruiz2!(Arows, Acols, Avals, Qrows, Qcols, Qvals, c, b, lvar, uvar, 
                       n_rows, n_cols, ϵ; max_iter = 100)
    n = length(Arows)
    T = eltype(Avals)
    d1, d2 = ones(T, n_rows), ones(T, n_cols)
    r_k, c_k = zeros(T, n_rows), zeros(T, n_cols)

    # scaling Q (symmetric)
    n_Q = length(Qrows)
    c_k = get_norm_rc!(c_k, Qcols, Qvals, n_cols, n_Q)
    convergence = maximum(abs.(one(T) .- c_k)) <= ϵ
    Qrows, Qcols, Qvals, d2 = mul_Q_D!(Qrows, Qcols, Qvals, d2, c_k, n_cols, n_Q)
    k = 1
    while !convergence && k < max_iter
        c_k = get_norm_rc!(c_k, Qcols, Qvals, n_cols, n_Q)
        convergence = maximum(abs.(one(T) .- c_k)) <= ϵ
        Qrows, Qcols, Qvals, d2 = mul_Q_D!(Qrows, Qcols, Qvals, d2, c_k, n_cols, n_Q)
        k += 1
    end
    
    for i=1:n
        Avals[i] *= d2[Acols[i]]
    end
    c .*= d2
    lvar ./= d2
    uvar ./= d2
    
    # scaling A
    r_k = get_norm_rc!(r_k, Arows, Avals, n_rows, n)
    convergence = maximum(abs.(one(T) .- r_k)) <= ϵ
    Arows, Avals, d1 = mul_A_D1!(Arows, Avals, d1, r_k, n_rows, n)
    k = 1
    while !convergence && k < max_iter
        r_k = get_norm_rc!(r_k, Arows, Avals, n_rows, n)
        convergence = maximum(abs.(one(T) .- r_k)) <= ϵ
        Arows, Avals, d1 = mul_A_D1!(Arows, Avals, d1, r_k, n_rows, n)
        k += 1
    end
    
    b .*= d1
    
    return Arows, Acols, Avals, Qrows, Qcols, Qvals, c, b, lvar, uvar, d1, d2
end


scaling_Ruiz2! (generic function with 1 method)

In [99]:
# presolve

function find_empty_rows(Arows, n_rows)
    n = length(Arows)
    rows = zeros(Int, n_rows)
    for i=1:n
        rows[Arows[i]] = one(Int)
    end
    empty_rows = zeros(Int, n_rows - sum(rows))
    c = 1
    for i=1:n_rows
        if rows[i] == zero(Int)
            empty_rows[c] = i
            c += 1
        end
    end
    return empty_rows
end

function rm_empty_rows!(empty_rows, Arows, Lcon, Ucon, n_rows)
    # si empty_rows non vide
    c = 1
    n = length(Arows)
    n_empty = length(empty_rows)
    T = eltype(Lcon)
    Lcon = deleteat!(Lcon, empty_rows)
    Ucon = deleteat!(Ucon, empty_rows)
    
    for i=1:n
        if Arows[i] > empty_rows[1]
            c = findlast(empty_rows .<= Arows[i])
            Arows[i] -= c    
        end
    end
    n_rows -= length(empty_rows)
    return Arows, Lcon, Ucon, n_rows
end


function find_singleton_rows(Arows, n_rows)
    nb_coef_rows = zeros(Int, n_rows) # nb nonzeros per row
    position_singleton = zeros(Int, n_rows) # row index of singleton rows
    for i=1:length(Arows)
        nb_coef_rows[Arows[i]] += 1
        position_singleton[Arows[i]] = i
    end
    singleton_rows = findall(isequal(1), nb_coef_rows)
    return singleton_rows, position_singleton[singleton_rows]
end


function rm_singleton_rows!(Arows, Acols, Avals, Lcon, Ucon, lvar, uvar, ifix, n_rows)
    # to do before rm_emtpy_rows
    T = eltype(Avals)
    singleton_rows, position_singleton = find_singleton_rows(Arows, n_rows)
    Arows_rm = zeros(T, length(singleton_rows))
    Acols_rm = zeros(T, length(singleton_rows))
    Avals_rm = zeros(T, length(singleton_rows))
    
    # x_i fixed
    for i=1:length(singleton_rows)
        if Avals[position_singleton[i]] > 0
            lvar[Acols[position_singleton[i]]] = max(lvar[Acols[position_singleton[i]]],
                                                    Lcon[Arows[position_singleton[i]]] / Avals[position_singleton[i]])
            uvar[Acols[position_singleton[i]]] = min(uvar[Acols[position_singleton[i]]],
                                                    Ucon[Acols[position_singleton[i]]] / Avals[position_singleton[i]]) 
        else
            lvar[Acols[position_singleton[i]]] = max(lvar[Acols[position_singleton[i]]],
                                                    Ucon[Arows[position_singleton[i]]] / Avals[position_singleton[i]])
            uvar[Acols[position_singleton[i]]] = min(uvar[Acols[position_singleton[i]]],
                                                    Lcon[Acols[position_singleton[i]]] / Avals[position_singleton[i]])
        end
        Arows_rm[i] = Arows[position_singleton[i]]
        Acols_rm[i] = Acols[position_singleton[i]]
        Avals_rm[i] = Avals[position_singleton[i]]
        if lvar[Acols[position_singleton[i]]] == uvar[Acols[position_singleton[i]]]
            push!(ifix, Acols[position_singleton[i]])
        end
    end
    
    ifix = unique!(ifix)
    
    #remove singleton rows
    position_singleton = sort!(position_singleton)
    Arows = deleteat!(Arows, position_singleton)
    Acols = deleteat!(Acols, position_singleton)
    Avals = deleteat!(Avals, position_singleton)
    
    return Arows, Acols, Avals, Lcon, Ucon, lvar, uvar, ifix, Arows_rm, Acols_rm, Avals_rm
end
    

function rm_ifix!(ifix, Qrows, Qcols, Qvals, c, c0, Arows, Acols, Avals, 
                  Lcon, Ucon, lvar, uvar, n_rows, n_cols)
    T = eltype(c)
    # get Qii
    diag_Q = zeros(T, n_cols)
    for i=1:length(Qvals)
        if Qrows[i] == Qcols[i]
            diag_Q[Qrows[i]] = Qvals[i]
        end
    end
    
    ifix = sort!(ifix) 

    # update c0, c
    Qji = zero(T)
    for i=1:length(ifix)
        c0 += c[ifix[i]] * lvar[ifix[i]] + diag_Q[ifix[i]] * lvar[ifix[i]]^2 / 2
        for j=1:n_cols
            Qji = zero(T)
            for k=1:length(Qvals)
                if (Qrows[k] == i && Qcols[k] == j) || (Qrows[k] == j && Qcols[k] == i)
                    Qji += Qvals[k]
                end
            end
            c[j] += lvar[ifix[i]] * Qji
        end
    end
    
    
    # remove columns in ifix
    ifix_cols_A = findall(x->x in ifix, Acols)
    ifix_cols_A = sort!(ifix_cols_A)
    for i=1:length(Acols)
        if i in ifix_cols_A
            Lcon[Arows[i]] -= Avals[i] * lvar[Acols[i]]
            Ucon[Arows[i]] -= Avals[i] * lvar[Acols[i]]
        end
    end
    Arows_rm_fix = Arows[ifix_cols_A]
    Acols_rm_fix = Acols[ifix_cols_A]
    Avals_rm_fix = Avals[ifix_cols_A]
    Arows = deleteat!(Arows, ifix_cols_A)
    Acols = deleteat!(Acols, ifix_cols_A)
    Avals = deleteat!(Avals, ifix_cols_A)
    
    for i=1:length(Acols)
        if Acols[i] > ifix[1]
            Acols[i] -= findlast(ifix .<= Acols[i])
        end
    end

    # remove rows and columns in ifix
    ifix_cols_Q = findall(x-> x in ifix, Qcols)

    
    Q_rm_idx = [] #unique(hcat(ifix_rows_Q, ifix_cols_Q))
    Qrows_rm_fix = Qrows[Q_rm_idx]
    Qcols_rm_fix = Qcols[Q_rm_idx]
    Qvals_rm_fix = Qvals[Q_rm_idx]
    for i=1:length(ifix_cols_Q)
        Qrows_rm_fix = push!(Qrows_rm_fix, splice!(Qrows, ifix_cols_Q[i]-i+1))
        Qcols_rm_fix = push!(Qcols_rm_fix, splice!(Qcols, ifix_cols_Q[i]-i+1))
        Qvals_rm_fix = push!(Qvals_rm_fix, splice!(Qvals, ifix_cols_Q[i]-i+1))
    end
    ifix_rows_Q = findall(x-> x in ifix, Qrows)
    for i=1:length(ifix_rows_Q)
        Qrows_rm_fix = push!(Qrows_rm_fix, splice!(Qrows, ifix_rows_Q[i]-i+1))
        Qcols_rm_fix = push!(Qcols_rm_fix, splice!(Qcols, ifix_rows_Q[i]-i+1))
        Qvals_rm_fix = push!(Qvals_rm_fix, splice!(Qvals, ifix_rows_Q[i]-i+1))
    end
    
    for i=1:length(Qcols)
        if  Qrows[i] > ifix[1]
            Qrows[i] -= findlast(ifix .<= Qrows[i]) 
        end
        if Qcols[i] > ifix[1]
            Qcols[i] -= findlast(ifix .<= Qcols[i])  
        end
    end
    
    c_rm_fix = c[ifix]
    x_rm_fix = lvar[ifix]
    c = deleteat!(c, ifix)
    lvar = deleteat!(lvar, ifix)
    uvar = deleteat!(uvar, ifix)
    n_cols -= length(ifix)
    
    return Qrows, Qcols, Qvals, c, c0, Arows, Acols, Avals, Lcon, Ucon, 
            lvar, uvar, n_cols, Arows_rm_fix, Acols_rm_fix, Avals_rm_fix,
            Qrows_rm_fix, Qcols_rm_fix, Qvals_rm_fix, c_rm_fix, x_rm_fix
end


rm_ifix! (generic function with 1 method)

In [ ]:
#postsolve

In [100]:
function presolve_QP!(qpdata)
    
    ifix = []
    for i=1:length(lvar)
        if lvar[i] == uvar[i]
            push!(ifix, i)
        end
    end
    qpdata.arows, qpdata.acols,
        qpdata.avals, qpdata.lcon, 
        qpdata.ucon, qpdata.lvar, 
        qpdata.uvar, ifix, 
        arows_rm, acols_rm, avals_rm = rm_singleton_rows!(qpdata.arows, qpdata.acols, 
                                                          qpdata.avals, qpdata.lcon, 
                                                          qpdata.ucon, qpdata.lvar, 
                                                          qpdata.uvar, ifix, 
                                                          qpdata.ncon)

    e_r = find_empty_rows(qpdata.arows, qpdata.ncon)

    if length(e_r) != 0
        qpdata.arows, qpdata.lcon, 
            qpdata.ucon, qpdata.ncon = rm_empty_rows!(e_r, qpdata.arows, 
                                                      qpdata.lcon, qpdata.ucon, 
                                                      qpdata.ncon)

    end
    
    if ifix != []
        qpdata.qrows, qpdata.qcols, 
            qpdata.qvals, qpdata.c, qpdata.c0, 
            qpdata.arows, qpdata.acols,
            qpdata.avals, qpdata.lcon, 
            qpdata.ucon, qpdata.lvar, 
            qpdata.uvar, qpdata.nvar = rm_ifix!(ifix, qpdata.qrows, 
                                                qpdata.qcols, qpdata.qvals, 
                                                qpdata.c, qpdata.c0, 
                                                qpdata.arows, qpdata.acols, 
                                                qpdata.avals, qpdata.lcon,
                                                qpdata.ucon, qpdata.lvar, 
                                                qpdata.uvar, qpdata.ncon,
                                                qpdata.nvar)
    end

    return qpdata
end

presolve_QP! (generic function with 1 method)

In [101]:
# tests pour le presolve
A = sparse([0. 0. 0.;
            0. 0. 0.;
            2. 2. 2. ; 
            0. 0. 0.;
            0. 0. 0.;
            0. 4. 0.;
            0. 2. 2.;
            0. 0. 0.;
            1. 1. 1.;
            0. 0. 0.;
            0. 0. 0.])
Lcon= [0.; 0. ;1. ; 0.; 0.; 40.; 2.; 0.; 2.; 0.; 0.]
Ucon= [0.; 0. ;5. ; 0.; 0.; 50.; 6.; 0.; 3.; 0.; 0.]

Q = sparse([1. 0. 0.;
            2. 3. 0.;
            0. 2. 4.])
c = [1. ;2.; 3.]
c0 = 1.
lvar = [-100; -2.; -Inf]
uvar = [Inf; 2; 18.]
ifix = [2]
ilow = []
iupp = [3]
irng = [2]
ifree = [1]
dropzeros!(A)
dropzeros!(Q)
Arows, Acols, Avals = findnz(A)
Qrows, Qcols, Qvals = findnz(Q)

n_rows = size(A)[1]
n_cols = size(A)[2]
Arows, Acols, Avals, Lcon, Ucon, lvar, uvar, ifix = rm_singleton_rows!(Arows, Acols, Avals, Lcon, Ucon,
                                                                       lvar, uvar, ifix, n_rows)
e_r = find_empty_rows(Arows, n_rows)
Arows, Lcon, Ucon, n_rows = rm_empty_rows!(e_r, Arows, Lcon, Ucon, n_rows)

# @code_warntype rm_ifix!(ifix, ilow, iupp, irng, Qrows, Qcols, Qvals, c, c0, 
#                                                   Arows, Acols, Avals, b, lvar, uvar, n_rows, n_cols)


Qrows, Qcols, Qvals, c, c0, Arows, Acols, 
    Avals, Lcon, Ucon, lvar, uvar, n_cols, 
    Arows_rm_fix, Acols_rm_fix, Avals_rm_fix,
    Qrows_rm_fix, Qcols_rm_fix, Qvals_rm_fix, 
    c_rm_fix, x_rm_fix = rm_ifix!(ifix, Qrows, Qcols, Qvals, c, c0, 
                                        Arows, Acols, Avals, Lcon, Ucon,lvar, uvar, n_rows, n_cols)


display(Matrix(sparse(Arows, Acols, Avals)))
display(Matrix(Symmetric(sparse(Qrows, Qcols, Qvals),:L)))
println(Lcon, Ucon, n_rows, n_cols)
println(lvar, uvar, ifix, c0)

3×2 Array{Float64,2}:
 2.0  2.0
 0.0  2.0
 1.0  1.0

2×2 Array{Float64,2}:
 1.0  0.0
 0.0  4.0

[1.0 0.0 0.0; 2.0 3.0 0.0; 0.0 2.0 4.0]
[-19.0, -18.0, -8.0][-15.0, -14.0, -7.0]32
[-100.0, -Inf][Inf, 18.0][2]171.0


In [14]:
function get_diag_sparseCSC(M; tri=:U)
    # get diagonal index of M.nzval
    # we assume all columns of M are non empty, and M triangular (:L or :U)
    @assert tri ==:U || tri == :L
    T = eltype(M)
    n = length(M.rowval)
    diagind = zeros(Int, M.m) # square matrix
    index = M.rowval[1] # 1
    if tri == :U
        for i=1:M.m
            diagind[i] = M.colptr[i+1] - 1       
        end
    else
        for i=1:M.m
            diagind[i] = M.colptr[i]        
        end
    end
    return diagind
end

function get_diag_sparseCOO(Qrows, Qcols, Qvals, n_cols)
    # get diagonal index of M.nzval
    # we assume all columns of M are non empty, and M triangular (:L or :U)
    T = eltype(Qvals)
    n = length(Qrows)
    diagval = zeros(T, n_cols)
    for i=1:n
        if Qrows[i] == Qcols[i]
            diagval[Qrows[i]] = Qvals[i]
        end
    end
    
    return diagval
end

get_diag_sparseCOO (generic function with 1 method)

In [15]:
function mehrotraPCQuadBounds(QM0; max_iter=100, ϵ_pdd=1e-8, ϵ_rb=1e-6, ϵ_rc=1e-6,
                              tol_Δx=1e-16, ϵ_μ=1e-9, max_time=60., scaling=true,
                              display=true, check_frontier=false)
    
    start_time = time()
    elapsed_time = 0.0
    QM = SlackModel(QM0)
    
    
    # get variables from QuadraticModel
    lvar, uvar = QM.meta.lvar, QM.meta.uvar
    n_cols = length(lvar)
    Oc = zeros(n_cols)
    ilow, iupp = [QM.meta.ilow; QM.meta.irng], [QM.meta.iupp; QM.meta.irng] # finite bounds index
    irng = QM.meta.irng
    ifix = QM.meta.ifix
    c = grad(QM, Oc)
    A = jac(QM, Oc)
    A = dropzeros!(A)
    T = eltype(A)
    Arows, Acols, Avals = findnz(A)
    n_rows, n_cols = size(A)
    @assert QM.meta.lcon == QM.meta.ucon # equality constraint (Ax=b)
    b = QM.meta.lcon
    Q = hess(QM, Oc)  # lower triangular
    Q = dropzeros!(Q)
    Qrows, Qcols, Qvals = findnz(Q)    
    c0 = obj(QM, Oc)
    


    if scaling
        Arows, Acols, Avals, Qrows, Qcols, Qvals, 
        c, b, lvar, uvar, d1, d2, d3 = scaling_Ruiz!(Arows, Acols, Avals, Qrows, Qcols, Qvals, 
                                                     c, b, lvar, uvar, n_rows, n_cols, T(1e-3))    
    end
    
    cNorm = norm(c)
    bNorm = norm(b)
    ANorm = norm(Avals)  # Frobenius norm after scaling; could be computed while scaling?
    QNorm = norm(Qvals)
    
    n_low, n_upp = length(ilow), length(iupp) # number of finite constraints

    # init regularization values
    ρ, δ = T(1e5*sqrt(eps())), T(1e6*sqrt(eps())) # 1e6, 1e-1 ok
#     ρ_min, δ_min = 1e0*T(sqrt(eps())), 100*T(sqrt(eps()))  
    ρ_min, δ_min = 1e-5*T(sqrt(eps())), 1e-5*T(sqrt(eps()))
    c_catch = zero(Int) # to avoid endless loop
    c_pdd = zero(Int) # avoid too small δ_min

    J_augmrows = vcat(Qcols, Acols, n_cols+1:n_cols+n_rows, 1:n_cols)
    J_augmcols = vcat(Qrows, Arows.+n_cols, n_cols+1:n_cols+n_rows, 1:n_cols)
    tmp_diag = -T(1.0e-7).*ones(T, n_cols)
    J_augmvals = vcat(-Qvals, Avals, δ*ones(n_rows), tmp_diag)
    J_augm = sparse(J_augmrows, J_augmcols, J_augmvals)
    diagind_J = get_diag_sparseCSC(J_augm)
    diag_Q = get_diag_sparseCOO(Qrows, Qcols, Qvals, n_cols)
    
    k = 0
    Δ_aff = zeros(T, n_cols+n_rows+n_low+n_upp)
    Δ_cc = zeros(T, n_cols+n_rows+n_low+n_upp)
    Δ = zeros(T, n_cols+n_rows+n_low+n_upp)
    Δ_xλ = zeros(T, n_cols+n_rows)

    x, λ, s_l, s_u, J_P, Qx, ATλ,
    x_m_lvar, uvar_m_x, Δ_xλ = @views starting_points(Qrows, Qcols, Qvals, Arows, Acols, Avals, 
                                                      b, c, lvar, uvar, ilow, iupp, QM.meta.irng, 
                                                      J_augm , n_rows, n_cols, Δ_xλ)

    Qx = mul_Qx_COO!(Qx, Qrows, Qcols, Qvals, x)
    ATλ = mul_ATλ_COO!(ATλ, Arows, Acols, Avals, λ)
    Ax = zeros(n_rows)
    Ax = mul_Ax_COO!(Ax, Arows, Acols, Avals, x)
    rb = Ax - b
    rc = -Qx + ATλ + s_l - s_u - c

    x_m_lvar .= @views x[ilow] .- lvar[ilow]
    uvar_m_x .= @views uvar[iupp] .- x[iupp]
    μ = @views compute_μ(x_m_lvar, uvar_m_x, s_l[ilow], s_u[iupp], n_low, n_upp)

    x_m_l_αΔ_aff = zeros(T, n_low) # x-lvar + αΔ_aff
    u_m_x_αΔ_aff = zeros(T, n_upp) # uvar-x + αΔ_aff
    s_l_αΔ_aff = zeros(T, n_low) # s_l + αΔ_aff
    s_u_αΔ_aff = zeros(T, n_upp) # s_l + αΔ_aff
    rxs_l, rxs_u = zeros(T, n_low), zeros(T, n_upp)
    ilow_pad, iupp_pad = zeros(Int, n_cols), zeros(Int, n_cols)
    ilow_pad[ilow] = 1:n_low
    iupp_pad[iupp] = 1:n_upp
    
    if check_frontier
        x_m_lvar_sup, uvar_m_x_sup = zeros(T, n_low), zeros(T, n_upp)
        s_l_sup, s_u_sup = zeros(T, n_low), zeros(T, n_upp)
    end
    
    # stopping criterion
    xTQx_2 = x' * Qx / 2
    cTx = c' * x
    pri_obj = xTQx_2 + cTx + c0
    dual_obj = b' * λ - xTQx_2 + view(s_l,ilow)'*view(lvar,ilow) -
                    view(s_u,iupp)'*view(uvar,iupp) +c0
    pdd = abs(pri_obj - dual_obj ) / (one(T) + abs(pri_obj) + abs(dual_obj))
    rcNorm, rbNorm = norm(rc), norm(rb)
    optimal = pdd < ϵ_pdd && rbNorm < ϵ_rb && rcNorm < ϵ_rc
    l_pdd = [pdd]

    n_Δx = zero(T)
    small_Δx, small_μ = false, μ < ϵ_μ
    Δt = time() - start_time
    tired = Δt > max_time

    # display
    if display == true
        @info log_header([:k, :pri_obj, :pdd, :rbNorm, :rcNorm, :n_Δx, :α_pri, :α_du, :μ],
                         [Int, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64],
                         hdr_override=Dict(:k => "iter", :pri_obj => "obj", :pdd => "rgap",
                                           :rbNorm => "‖rb‖", :rcNorm => "‖rc‖",
                                           :n_Δx => "‖Δx‖"))
        @info log_row(Any[k, pri_obj, pdd, rbNorm, rcNorm, n_Δx, zero(T), zero(T), μ])
    end

    while k<max_iter && !optimal && !tired # && !small_μ && !small_μ

            # Affine scaling direction
        tmp_diag .= -ρ
        tmp_diag[ilow] .-= @views s_l[ilow] ./ x_m_lvar
        tmp_diag[iupp] .-= @views s_u[iupp] ./ uvar_m_x
        
#         J_augmvals[end-n_cols+1:end] = tmp_diag
#         J_augm = sparse(J_augmrows, J_augmcols, J_augmvals)
        
        J_augm.nzval[view(diagind_J,1:n_cols)] .= @views tmp_diag .- diag_Q
        J_augm.nzval[view(diagind_J, n_cols+1:n_rows+n_cols)] .= δ
         
#         J_fact = ldl_factorize!(Symmetric(J_augm, :U), J_P)
        
        J_fact = try ldl_factorize!(Symmetric(J_augm, :U), J_P)
        catch 
            println("error ", k)
            if c_pdd == 0
                δ *= 1e7
                δ_min *= 1e7
                ρ *= 1e5
                ρ_min *= 1e5
            else
                δ *= 1e8
                δ_min *= 1e8
                ρ *= 1e5
                ρ_min *= 1e5
            end
            c_catch += 1
            tmp_diag .= -ρ
            tmp_diag[ilow] .-= @views s_l[ilow] ./ x_m_lvar
            tmp_diag[iupp] .-= @views s_u[iupp] ./ uvar_m_x
            J_augm.nzval[view(diagind_J,1:n_cols)] .= @views tmp_diag .- diag_Q
            J_augm.nzval[view(diagind_J, n_cols+1:n_rows+n_cols)] .= δ
            J_fact = ldl_factorize!(Symmetric(J_augm, :U), J_P)
        end
            
        if c_catch >= 3
            break
        end

        Δ_aff = solve_augmented_system_aff!(J_fact, Δ_aff, Δ_xλ, rc, rb, x_m_lvar, uvar_m_x,
                                            s_l, s_u, ilow, iupp,  n_cols, n_rows, n_low)

        α_aff_pri = @views compute_α_primal(x, Δ_aff[1:n_cols], lvar, uvar)
        α_aff_dual_l = @views compute_α_dual(s_l[ilow], Δ_aff[n_rows+n_cols+1: n_rows+n_cols+n_low])
        α_aff_dual_u = @views compute_α_dual(s_u[iupp], Δ_aff[n_rows+n_cols+n_low+1:end])

        # alpha_aff_dual_final is the min of the 2 alpha_aff_dual
        α_aff_dual_final = min(α_aff_dual_l, α_aff_dual_u)
        
        x_m_l_αΔ_aff .= @views x_m_lvar .+ α_aff_pri .* Δ_aff[1:n_cols][ilow]
        u_m_x_αΔ_aff .= @views uvar_m_x .- α_aff_pri .* Δ_aff[1:n_cols][iupp]
        s_l_αΔ_aff .= @views s_l[ilow] .+ α_aff_dual_final .* Δ_aff[n_rows+n_cols+1: n_rows+n_cols+n_low]
        s_u_αΔ_aff .= @views s_u[iupp] .+ α_aff_dual_final .*  Δ_aff[n_rows+n_cols+n_low+1: end] 

        μ_aff = compute_μ(x_m_l_αΔ_aff, u_m_x_αΔ_aff, s_l_αΔ_aff, s_u_αΔ_aff,
                          n_low, n_upp)

        σ = (μ_aff / μ)^3

        # corrector and centering step

        Δ_cc = solve_augmented_system_cc!(J_fact, Δ_cc, Δ_xλ , Δ_aff, σ, μ,x_m_lvar, uvar_m_x,
                                          rxs_l, rxs_u, s_l, s_u, ilow, iupp, n_cols, n_rows, n_low)


        Δ .= Δ_aff .+ Δ_cc # final direction

        α_pri = @views compute_α_primal(x, Δ[1:n_cols], lvar, uvar)
        α_dual_l = @views compute_α_dual(s_l[ilow], Δ[n_rows+n_cols+1: n_rows+n_cols+n_low])
        α_dual_u = @views compute_α_dual(s_u[iupp], Δ[n_rows+n_cols+n_low+1: end])

        α_dual_final = min(α_dual_l, α_dual_u)

        if check_frontier
            α_pri, α_dual_final = check_frontier!(T(1e-3), x, x_m_lvar, uvar_m_x, s_l, s_u,
                                                  Δ, α_pri, α_dual_final, 
                                                  x_m_lvar_sup, uvar_m_x_sup, s_l_sup, s_u_sup,
                                                  lvar, uvar, ilow, iupp,
                                                  ilow_pad, iupp_pad, n_low, n_upp, n_rows, n_cols)
        end
        # new parameters
        x .= @views x .+ α_pri .* Δ[1:n_cols]
        λ .= @views λ .+ α_dual_final .* Δ[n_cols+1: n_rows+n_cols]
        s_l[ilow] .= @views s_l[ilow] .+ α_dual_final .* Δ[n_rows+n_cols+1: n_rows+n_cols+n_low]
        s_u[iupp] .= @views s_u[iupp] .+ α_dual_final .* Δ[n_rows+n_cols+n_low+1: end]
        n_Δx = @views α_pri * norm(Δ[1:n_cols])
        x_m_lvar .= @views x[ilow] .- lvar[ilow]
        uvar_m_x .= @views uvar[iupp] .- x[iupp]
        
        if zero(T) in x_m_lvar
            for i=1:n_low
                if x_m_lvar[i] == zero(T)
                    x_m_lvar[i] = eps()
                end
            end
        end
        if zero(T) in uvar_m_x
            for i=1:n_upp
                if uvar_m_x[i] == zero(T)
                    uvar_m_x[i] = eps()
                end
            end
        end
        
        μ = @views compute_μ(x_m_lvar, uvar_m_x, s_l[ilow], s_u[iupp],
                             n_low, n_upp)
        
        
        Qx = mul_Qx_COO!(Qx, Qrows, Qcols, Qvals, x)
        xTQx_2 =  x' * Qx / 2
        ATλ = mul_ATλ_COO!(ATλ, Arows, Acols, Avals, λ)
        Ax = mul_Ax_COO!(Ax, Arows, Acols, Avals, x)
        cTx = c' * x
        pri_obj = xTQx_2 + cTx + c0
        dual_obj = b' * λ - xTQx_2 + view(s_l,ilow)'*view(lvar,ilow) -
                    view(s_u,iupp)'*view(uvar,iupp) +c0

        rb .= Ax .- b
        rc .= ATλ .-Qx .+ s_l .- s_u .- c

        # update stopping criterion values:

        pdd = abs(pri_obj - dual_obj ) / (one(T) + abs(pri_obj))
        rcNorm, rbNorm = norm(rc), norm(rb)
        xNorm = norm(x)
        λNorm = norm(λ)
        optimal = pdd < ϵ_pdd && rbNorm < ϵ_rb * max(1, bNorm + ANorm * xNorm) && 
                    rcNorm < ϵ_rc * max(1, cNorm + QNorm * xNorm + ANorm * λNorm)
        small_Δx, small_μ = n_Δx < tol_Δx, μ < ϵ_μ
        k += 1
        
        push!(l_pdd, pdd)
        if μ < 1e-40 && c_pdd == 0
            println("mu  ", k)
#             δ_min /= 1e3
#             δ /= 1e3
#             c_pdd += 3
        elseif k > 10  && std(l_pdd[end-5:end]./mean(l_pdd[end-5:end])) < 1e-2 && c_pdd < 3
            println("pdd  ", k)
            δ_min /= 1e1
            δ /= 1e1
            c_pdd += 1
        elseif μ < 1e-50 && c_pdd == 3
#             println("mu2  ", k)
#             δ_min /= 1e2
#             δ /= 1e2
#             c_pdd += 1
        end
        
        
        if δ >= δ_min
            δ /= 10
            #J_augmvals[end-n_cols-n_rows+1:end-n_cols] .= δ
        end
        if ρ >= ρ_min
            ρ /= 10
        end

        Δt = time() - start_time
        tired = Δt > max_time

        if display == true
            @info log_row(Any[k, pri_obj, pdd, rbNorm, rcNorm, n_Δx, α_pri, α_dual_final, μ])
        end

#         if k==98
#             println("iter", k)
#             println("min |x+αΔ_aff-lvar| = ", minimum(abs.(x_m_l_αΔ_aff)))
#             println("min |x-lvar| = ", minimum(abs.(x_m_lvar)))
#             println("α_pri = ", α_pri)
#             idx_pb = findall(isequal(0.), abs.(x_m_lvar))
#             for i in idx_pb
#                 println("   i = ", i, "  xi = ", x[i], "   lvari = " ,lvar[i])
#             end
#         end
    end

    if k>= max_iter
        status = :max_iter
    elseif tired
        status = :max_time
    elseif optimal
        status = :acceptable
    else
        status = :unknown
    end

    if scaling
        x .*= d2 .* d3
        for i=1:length(Qrows)
            Qvals[i] /= d2[Qrows[i]] * d2[Qcols[i]] * d3[Qrows[i]] * d3[Qcols[i]]
        end
        Qx = mul_Qx_COO!(Qx, Qrows, Qcols, Qvals, x)
        xTQx_2 =  x' * Qx / 2
        for i=1:length(Arows)
            Avals[i] /= d1[Arows[i]] * d2[Acols[i]] * d3[Acols[i]]
        end
        λ .*= d1
        ATλ = mul_ATλ_COO!(ATλ, Arows, Acols, Avals, λ)
        Ax = mul_Ax_COO!(Ax, Arows, Acols, Avals, x)
        b ./= d1
        c ./= d2 .* d3
        cTx = c' * x
        pri_obj = xTQx_2 + cTx + c0
        lvar .*= d2 .* d3
        uvar .*= d2 .* d3
        dual_obj = b' * λ - xTQx_2 + view(s_l,ilow)'*view(lvar,ilow) -
                    view(s_u,iupp)'*view(uvar,iupp) +c0
        
        s_l ./= d2 .* d3
        s_u ./= d2 .* d3
        rb .= Ax .- b
        rc .= ATλ .-Qx .+ s_l .- s_u .- c
        rcNorm, rbNorm = norm(rc), norm(rb)
    end
    
    elapsed_time = time() - start_time

    stats = GenericExecutionStats(status, QM, solution = x,
                                  objective = pri_obj ,
                                  dual_feas = rcNorm,
                                  primal_feas = rbNorm,
                                  solver_specific = Dict(:multipliers => λ,
                                                         :multipliers_L => s_l,
                                                         :multipliers_U => s_u),
                                  iter = k,
                                  elapsed_time=elapsed_time)
    return stats
end



mehrotraPCQuadBounds (generic function with 1 method)

### bibliothèque QuadraticModels

optimize $c_0 + c'x + \frac{1}{2} x'Hx ~~~~$ s.t. $~~L \le Ax \le U$ and $l \le x \le u$ 

Ici L = lcon, U = ucon, l = lvar, u = uvar

In [16]:
# probleme1
Q = [6. 2. 1.
     2. 5. 2.
     1. 2. 4.]
c = [-8.; -3; -3]
c0 = 0.
A = [1. 0. 1.
    0. 1. 1.]
b = [0.; 3]
lvar = [0.;0;0]
uvar = [Inf; Inf; Inf]
lcon = b
ucon = b

x01 = [1.; 2.; 3.];

In [17]:
QM = QuadraticModel(c, Q, A=A, lcon=lcon, ucon=ucon, lvar=lvar, uvar=uvar, x0=x01, c0=c0, name="QM1")
SM = SlackModel(QM)
typeof(SM)

QuadraticModel

In [18]:
stats_mpc1 =  mehrotraPCQuadBounds(SM, scaling=true, presolve=false)
println(stats_mpc1)

MethodError: MethodError: no method matching mehrotraPCQuadBounds(::QuadraticModel; scaling=true, presolve=false)
Closest candidates are:
  mehrotraPCQuadBounds(::Any; max_iter, ϵ_pdd, ϵ_rb, ϵ_rc, tol_Δx, ϵ_μ, max_time, scaling, display, check_frontier) at In[15]:5 got unsupported keyword argument "presolve"

In [19]:
#pb verif

QM_verif = QuadraticModel(c*(-1), Q, A=A*(-1), lcon=lcon, ucon=ucon, lvar=-uvar, uvar=lvar, x0=x01, c0=c0, name="QM1")
SM_verif = SlackModel(QM_verif)
stats_mpc_verif =  mehrotraPCQuadBounds(SM_verif);
println(stats_mpc_verif)

┌ Info:   iter       obj      rgap      ‖rb‖      ‖rc‖      ‖Δx‖     α_pri      α_du         μ  
└ @ Main In[15]:114
┌ Info:      0   1.4e+02   1.0e+00   9.1e+00   9.1e+00   0.0e+00   0.0e+00   0.0e+00   3.9e+01
└ @ Main In[15]:119
┌ Info:      1   1.5e+01   1.1e+00   1.0e+00   1.8e+00   9.9e+00   9.0e-01   1.0e+00   4.7e+00
└ @ Main In[15]:287
┌ Info:      2   1.3e+01   5.0e-03   8.7e-03   2.4e-04   1.6e+00   1.0e+00   1.0e+00   3.7e-02
└ @ Main In[15]:287
┌ Info:      3   1.3e+01   7.6e-06   1.2e-05   3.2e-06   2.2e-02   1.0e+00   1.0e+00   3.9e-05
└ @ Main In[15]:287


Generic Execution stats
  status: solved to within acceptable tolerances
  objective value: 13.499999826100257
  primal feasibility: 1.3352583074087876e-8
  dual feasibility: 7.145226623998341e-9
  solution: [-3.668871827442906e-9  -2.9999999852654033  -3.417720798623904e-9]
  iterations: 4
  elapsed time: 4.82699990272522
  solver specific:
    multipliers_U: [

┌ Info:      4   1.3e+01   8.5e-09   1.3e-08   3.2e-09   3.3e-05   1.0e+00   1.0e+00   3.9e-08
└ @ Main In[15]:287


15.673187269433072  1.0101865786121473e-8  8.673187334979994]
    multipliers_L: [0.0  0.0  0.0]
    multipliers: [-17.673187271890733  11.99999992354133]


In [20]:
### pb verif 2
c2 = [1.; 0; 0]
A2 = Matrix([1., 1, 1]')
b2 = [1.];
QM_verif2 = QuadraticModel(c2, zeros(3,3), A=A2, lcon=b2, ucon=b2, lvar=[0.;0;0], 
                uvar=[Inf; Inf; Inf], x0=x01, c0=c0, name="QMverfi2")
SM_verif2 = SlackModel(QM_verif2)
stats_mpc_verif2 =  mehrotraPCQuadBounds(SM_verif2);
println(stats_mpc_verif2)

Generic Execution stats
  status: solved to within acceptable tolerances
  objective value: 2.396822033142898e-10
  primal feasibility: 1.6580592454573662e-10
  dual feasibility: 2.2159359856144903e-11
  solution: [2.396822033142898e-10  0.49999999979725596  0.49999999979725596]
  iterations: 4
  elapsed time: 0.0
  solver specific:
    multipliers_U: [0.0  0.0  0.0]
    

┌ Info:   iter       obj      rgap      ‖rb‖      ‖rc‖      ‖Δx‖     α_pri      α_du         μ  
└ @ Main In[15]:114
┌ Info:      0   2.0e+00   6.7e-01   5.0e+00   2.9e-01   0.0e+00   0.0e+00   0.0e+00   1.0e+00
└ @ Main In[15]:119
┌ Info:      1   5.4e-02   1.5e-01   1.5e-03   4.3e-03   2.9e+00   1.0e+00   1.0e+00   5.1e-02
└ @ Main In[15]:287
┌ Info:      2   2.4e-04   3.4e-04   1.5e-04   1.9e-05   6.6e-02   1.0e+00   1.0e+00   1.1e-04
└ @ Main In[15]:287
┌ Info:      3   2.4e-07   3.4e-07   1.6e-07   2.2e-08   3.6e-04   1.0e+00   1.0e+00   1.1e-07
└ @ Main In[15]:287
┌ Info:      4   2.4e-10   3.4e-10   1.7e-10   2.2e-11   3.7e-07   1.0e+00   1.0e+00   1.1e-10
└ @ Main In[15]:287


multipliers_L: [1.0000000000763423  9.7600665859545e-11  9.7600665859545e-11]
    multipliers: [-9.846765661266674e-11]


# Lecture des donnees .SIF

In [21]:

function createQuadraticModel(qpdata; name="qp_pb")
    return QuadraticModel(qpdata.c, qpdata.qrows, qpdata.qcols, qpdata.qvals,
            Arows=qpdata.arows, Acols=qpdata.acols, Avals=qpdata.avals, 
            lcon=qpdata.lcon, ucon=qpdata.ucon, lvar=qpdata.lvar, uvar=qpdata.uvar,
            c0=qpdata.c0, name=name)
end

function displayQuadraticModel(QM)
    #println("A = ", Matrix(jac(QM, QM.meta.x0)))
    #println("Q = ", Matrix(hess(QM, QM.meta.x0)))
    println("lvar = ", QM.meta.lvar)
    println("uvar = ", QM.meta.uvar)
    println("x0 = ", QM.meta.x0)
    #println("lcon = ", QM.meta.lcon)
    #println("ucon = ", QM.meta.ucon)
end

displayQuadraticModel (generic function with 1 method)

# Benchmark

In [22]:
path_pb = "C:\\Users\\Geoffroy Leconte\\Documents\\cours\\TFE\\code\\problemes_netlib"
path_pb_QP = "C:\\Users\\Geoffroy Leconte\\Documents\\cours\\TFE\\code\\problemes_marosmeszaros"
afiro = string(path_pb, "\\AFIRO.SIF")

"C:\\Users\\Geoffroy Leconte\\Documents\\cours\\TFE\\code\\problemes_netlib\\AFIRO.SIF"

In [23]:
qpdata2 = readqps(afiro)  # obj   -4.6475314286E+02
# presolve_QP!(qpdata2)
QM2 = createQuadraticModel(qpdata2)
SM2 = SlackModel(QM2);
SM2.meta

┌ Info: Using 'AFIRO' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'COST' as objective (l. 47)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'B' as RHS (l. 96)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


  Problem name: qp_pb-slack
   All variables: ████████████████████ 51     All constraints: ████████████████████ 27    
            free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                 free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: █████████████⋅⋅⋅⋅⋅⋅⋅ 32               lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           upper: ████████⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 19               upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
         low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0              low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ████████████████████ 27    
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: (100.00% sparsity)   0               linear: ████████████████████ 27    
                                                    nonlinear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                                                         nnzj: ( 92.59% sparsity)   102   



In [24]:
stats_mpc2 =  mehrotraPCQuadBounds(SM2, max_iter=20)
println(stats_mpc2)

Generic Execution stats
  status: solved to within acceptable tolerances
  objective value: -464.7531428149319
  primal feasibility: 4.612753923448528e-13
  dual feasibility: 2.9172534023306517e-13
  solution: [79.99999999649363  25.499999993863284  54.50000000263025  84.79999999628329 ⋯ 90.05025726564638]
  iterations: 8

┌ Info:   iter       obj      rgap      ‖rb‖      ‖rc‖      ‖Δx‖     α_pri      α_du         μ  
└ @ Main In[15]:114
┌ Info:      0   6.0e+02   1.0e+00   6.9e+02   1.2e+01   0.0e+00   0.0e+00   0.0e+00   1.6e+02
└ @ Main In[15]:119
┌ Info:      1   7.1e-01   5.6e+02   5.7e+01   1.7e+00   4.0e+02   9.2e-01   8.3e-01   2.8e+01
└ @ Main In[15]:287
┌ Info:      2  -1.6e+02   1.6e+00   6.2e-03   5.7e-01   3.4e+02   1.0e+00   6.7e-01   8.5e+00
└ @ Main In[15]:287
┌ Info:      3  -4.0e+02   6.5e-01   3.6e-03   4.0e-02   5.1e+02   4.0e-01   9.5e-01   5.7e+00
└ @ Main In[15]:287
┌ Info:      4  -4.5e+02   1.1e-01   2.6e-04   9.4e-03   1.0e+02   9.3e-01   7.7e-01   1.2e+00
└ @ Main In[15]:287
┌ Info:      5  -4.6e+02   2.6e-02   9.1e-05   7.2e-06   3.2e+01   6.6e-01   1.0e+00   2.3e-01
└ @ Main In[15]:287
┌ Info:      6  -4.6e+02   1.5e-04   3.6e-07   1.5e-07   1.0e+01   1.0e+00   1.0e+00   1.3e-03
└ @ Main In[15]:287
┌ Info:      7  -4.6e+02   1.5e-07   3.6e-10   2.7e-10   7.8e-02   1.0e+00   1


  elapsed time: 0.9200000762939453
  solver specific:
    multipliers_U: [0.0  0.0  0.0  0.0 ⋯ 5.330327381335236e-12]
    multipliers_L: [1.3012198608341523e-11  5.562434970030833e-11  1.9816727060243273e-11  1.2275681343898081e-11 ⋯ 0.0]
    multipliers: [-0.6285714286813776  1.0676608700534942e-10  -0.34477142859027077  -0.22857142862577257 ⋯ -5.3384826148500124e-12]


In [25]:
@benchmark mehrotraPCQuadBounds(SM2,  display=false)

BenchmarkTools.Trial: 
  memory estimate:  249.80 KiB
  allocs estimate:  2715
  --------------
  minimum time:     496.701 μs (0.00% GC)
  median time:      510.000 μs (0.00% GC)
  mean time:        544.342 μs (2.80% GC)
  maximum time:     4.107 ms (71.72% GC)
  --------------
  samples:          9132
  evals/sample:     1

In [35]:
# problem 3   kb2    obj  -1.7499001299E+03
kb2 = string(path_pb, "\\KB2.SIF")
qpdata3 = readqps(kb2)
QM3 = createQuadraticModel(qpdata3)
SM3 = SlackModel(QM3)
SM3.meta

┌ Info: Using 'KB2' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'FAT7..J.' as objective (l. 20)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using '77BOUND' as BOUNDS (l. 227)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:622


  Problem name: qp_pb-slack
   All variables: ████████████████████ 68     All constraints: ████████████████████ 43    
            free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                 free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: ██████████████⋅⋅⋅⋅⋅⋅ 47               lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           upper: ████⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 12               upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
         low/upp: ███⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 9              low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ████████████████████ 43    
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: (100.00% sparsity)   0               linear: ████████████████████ 43    
                                                    nonlinear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                                                         nnzj: ( 89.30% sparsity)   313   



In [46]:
stats_mpc3 = mehrotraPCQuadBounds(SM3, scaling=true);
println(stats_mpc3)

Generic Execution stats
  status: solved to within acceptable tolerances
  objective value: -1749.9001298660821
  primal feasibility: 3.258851877166786e-7
  dual feasibility: 1.2773293074861207e-7
  solution: [0.8118234752348552  9.237300764043926e-10  1.5940599836501506e-9  7.70490065758585e-9 ⋯ -783.4591270076422]
  iterations: 16
  elapsed time: 0.00800013542175293
  solver specific:
    multipliers_U: [0.0  1.047769616923197e-11  0.0  0.0 ⋯ 1.236851136307068e-13]
    multipliers_L: [3.86992458678703e-11  0.06381155465707078  0.04896147780940663  0.10208310939823424 ⋯ 0.0]
    multipliers: [17.269208277354323  17.214101642892764  16.942045548979742  16.659838411836063 ⋯ -1.2368492821884643e-13]


┌ Info:   iter       obj      rgap      ‖rb‖      ‖rc‖      ‖Δx‖     α_pri      α_du         μ  
└ @ Main In[35]:155
┌ Info:      0  -1.0e+03   9.1e-01   4.2e+02   2.9e+02   0.0e+00   0.0e+00   0.0e+00   7.0e+02
└ @ Main In[35]:160
┌ Info:      1  -4.1e+02   4.2e+01   1.3e+02   1.4e+02   1.0e+02   6.9e-01   5.0e-01   4.3e+02
└ @ Main In[35]:327
┌ Info:      2  -2.4e+02   2.4e+01   1.2e+01   4.2e+01   1.1e+02   9.1e-01   7.1e-01   1.4e+02
└ @ Main In[35]:327
┌ Info:      3  -3.3e+02   8.2e+00   3.1e-02   1.9e+01   8.0e+01   1.0e+00   5.4e-01   7.0e+01
└ @ Main In[35]:327
┌ Info:      4  -4.4e+02   4.9e-01   2.4e-03   3.8e+00   7.3e+01   1.0e+00   8.0e-01   1.2e+01
└ @ Main In[35]:327
┌ Info:      5  -1.1e+03   3.7e-01   1.9e-03   2.6e+00   4.8e+02   2.2e-01   3.1e-01   8.8e+00
└ @ Main In[35]:327
┌ Info:      6  -1.5e+03   1.9e-01   1.6e-03   2.1e+00   3.2e+02   1.4e-01   2.0e-01   1.1e+01
└ @ Main In[35]:327
┌ Info:      7  -1.6e+03   7.6e-03   8.1e-04   1.0e+00   1.4e+02   5.0e-01   5

In [517]:
@benchmark mehrotraPCQuadBounds(SM3, display = false)

BenchmarkTools.Trial: 
  memory estimate:  603.11 KiB
  allocs estimate:  4598
  --------------
  minimum time:     912.099 μs (0.00% GC)
  median time:      949.450 μs (0.00% GC)
  mean time:        1.007 ms (2.98% GC)
  maximum time:     6.311 ms (79.97% GC)
  --------------
  samples:          4962
  evals/sample:     1

In [36]:
# problems AGG2, BNL2, FFFFF800, CYCLE, D2Q06C, FIT1P, FIT2P, CRE-B, SHIP04S, SHIP08L, LOTFI, MODSZK1
# CONT-050, CONT-100, CONT-101, CONT-201, CONT-300, BOYD2, LISWET1, POWELL20, QSEBA,QGROW22,QFFFFF80, QSHIP12L
# QGFRDXPN, QPCBOEI2, CVXQP3_L
pb4 = string(path_pb_QP, "\\QSEBA.SIF")
qpdata4 = readqps(pb4, mpsformat = :fixed)
QM4 = createQuadraticModel(qpdata4)
SM4 = SlackModel(QM4)
SM4.meta

┌ Info: Using 'QSEBA' as NAME (l. 1)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using '00000000' as objective (l. 17)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'RHS1' as RHS (l. 3173)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'RANGE1' as RANGES (l. 3179)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:552
┌ Info: Using 'BND1' as BOUNDS (l. 3184)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:622


false


  Problem name: qp_pb-slack
   All variables: ████████████████████ 1036   All constraints: ████████████████████ 515   
            free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                 free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: ███████████⋅⋅⋅⋅⋅⋅⋅⋅⋅ 522              lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
         low/upp: ██████████⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 514            low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ████████████████████ 515   
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: ( 99.88% sparsity)   646             linear: ████████████████████ 515   
                                                    nonlinear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                                                         nnzj: ( 99.18% sparsity)   4360  



In [235]:
stats_mpc4 = mehrotraPCQuadBounds(SM4, max_iter=50, max_time=600, scaling=false, 
                                  check_frontier=true)
println(stats_mpc4)

┌ Info:   iter       obj      rgap      ‖rb‖      ‖rc‖      ‖Δx‖     α_pri      α_du         μ  
└ @ Main In[179]:155
┌ Info:      0   3.9e+09   1.0e+00   1.5e+03   2.0e+07   0.0e+00   0.0e+00   0.0e+00   7.4e-04
└ @ Main In[179]:160
┌ Info:      1   3.9e+09   2.0e+00   1.5e+03   2.0e+07   8.4e+00   9.6e-03   2.3e-01   1.2e-03
└ @ Main In[179]:328
┌ Info:      2   3.9e+09   2.0e+00   1.5e+03   2.0e+07   7.2e-01   2.2e-04   7.5e-05   1.2e-03
└ @ Main In[179]:328
┌ Info:      3   3.9e+09   2.0e+00   1.5e+03   2.0e+07   4.1e-01   4.0e-05   8.1e-07   1.2e-03
└ @ Main In[179]:328
┌ Info:      4   3.9e+09   2.0e+00   1.5e+03   2.0e+07   3.6e-01   2.4e-05   3.0e-05   1.3e-03
└ @ Main In[179]:328
┌ Info:      5   3.9e+09   2.0e+00   1.5e+03   2.0e+07   1.0e+00   2.6e-06   4.9e-07   2.7e-03
└ @ Main In[179]:328
┌ Info:      6   3.9e+09   2.0e+00   1.5e+03   2.0e+07   6.4e-01   2.5e-06   1.4e-07   2.9e-03
└ @ Main In[179]:328
┌ Info:      7   3.9e+09   2.0e+00   1.5e+03   2.0e+07   3.0e-01   6.9

pdd  11


┌ Info:     10   3.9e+09   2.0e+00   1.5e+03   2.0e+07   4.5e-01   2.4e-05   2.6e-05   7.8e-01
└ @ Main In[179]:328
┌ Info:     11   3.9e+09   2.0e+00   1.5e+03   2.0e+07   9.2e-01   5.4e-06   3.2e-06   9.0e-01
└ @ Main In[179]:328


pdd  12


┌ Info:     12   3.9e+09   2.0e+00   1.5e+03   2.0e+07   1.2e+00   3.9e-05   2.8e-05   1.5e+00
└ @ Main In[179]:328


pdd  13


┌ Info:     13   3.9e+09   2.0e+00   1.5e+03   2.0e+07   9.9e-01   3.2e-05   9.9e-05   4.3e+00
└ @ Main In[179]:328
┌ Info:     14   3.9e+09   2.0e+00   1.5e+03   2.0e+07   1.6e+00   3.2e-05   6.2e-06   4.6e+00
└ @ Main In[179]:328
┌ Info:     15   3.9e+09   2.0e+00   1.5e+03   2.0e+07   8.1e-01   2.8e-05   9.7e-05   1.4e+01
└ @ Main In[179]:328
┌ Info:     16   3.9e+09   2.0e+00   1.5e+03   2.0e+07   5.3e-01   9.0e-06   5.0e-05   3.3e+01
└ @ Main In[179]:328
┌ Info:     17   3.9e+09   2.1e+00   1.5e+03   2.0e+07   1.8e-01   1.8e-06   4.3e-05   7.6e+01
└ @ Main In[179]:328
┌ Info:     18   3.9e+09   2.1e+00   1.5e+03   2.0e+07   5.7e-01   3.3e-06   1.4e-05   1.1e+02
└ @ Main In[179]:328
┌ Info:     19   3.9e+09   2.1e+00   1.5e+03   2.0e+07   1.3e+00   1.2e-06   1.1e-06   1.1e+02
└ @ Main In[179]:328
┌ Info:     20   3.9e+09   2.1e+00   1.5e+03   2.0e+07   1.3e+00   1.5e-07   9.0e-09   1.1e+02
└ @ Main In[179]:328
┌ Info:     21   3.9e+09   2.1e+00   1.5e+03   2.0e+07   4.4e-01   9.2e-

Generic Execution stats
  status: maximum iteration
  objective value: 3.8614437990522833e9
  primal feasibility: 1483.8651429280517
  dual feasibility: 1.99015738261209e7
  solution: [4.6571203123079625  0.4093403162806168  4.970366647413824  1.0439777930004575 ⋯ 4.996639023884199]
  iterations: 50
  elapsed time: 199.6529998779297
  solver specific:
    multipliers_U: [418.2400160818596  235.06811746662848  448.3856254792934  252.67307616208504 ⋯ 451.38365548702416]
    multipliers_L: [501.5303402532374  7718.243148744089  464.9962392196071  2345.8226843925045 ⋯ 461.20754173298644]
    multipliers: [-2175.4313299772375  -4705.493974759656  -33.114958285834675  13220.419154493646 ⋯ -6.616398569981912e11]


┌ Info:     49   3.9e+09   3.0e+04   1.5e+03   2.0e+07   2.0e+00   8.4e-06   2.0e-05   3.2e+07
└ @ Main In[179]:328
┌ Info:     50   3.9e+09   3.0e+04   1.5e+03   2.0e+07   1.2e+00   1.4e-07   4.3e-07   3.3e+07
└ @ Main In[179]:328


# Tests sur tous les problèmes

In [49]:
# T = typeof(SM2.meta)
# for (name, typ) in zip(fieldnames(T), T.types)
#     println("type of the fieldname $name is $typ")
# end

In [183]:
problems = []
i_max = 11
i = 1
for file_name in readdir(path_pb)
    if file_name[end-3:end] == ".SIF" && !(file_name in ["80BAU3B.SIF" ; "BORE3D.SIF"; 
                                                        "CAPRI.SIF"; "CZPROB.SIF"; 
                                                        "ETAMACRO.SIF"; "FINNIS.SIF";
                                                        "FORPLAN.SIF"; "GREENBEA.SIF";
                                                        "GREENBEB.SIF"; "MAROS.SIF";
                                                        "NESM.SIF"; "PEROLD.SIF";
                                                         "PILOT-JA.SIF"; "PILOT-WE.SIF";
                                                         "PILOT.SIF"; "PILOT4.SIF";
                                                         "PILOT87.SIF"; "PILOTNOV.SIF";
                                                          "RECIPELP.SIF"; "SHELL.SIF";
                                                         "SIERRA.SIF"; "STAIR.SIF";
                                                         "STANDATA.SIF"; "STANDGUB.SIF";
                                                        "STANDMPS.SIF"; "TUFF.SIF";
                                                        "VTP-BASE.SIF"; "DTOC3.SIF";
                                                         "HS35MOD.SIF";"QBORE3D.SIF";
                                                        "QCAPRI.SIF"; "QETAMACR.SIF";
                                                          "QFORPLAN.SIF"; "QPCSTAIR.SIF";
                                                        "QPCSTAIR.SIF"; "QPILOTNO.SIF";
                                                        "QRECIPE.SIF"; "QSHELL.SIF";
                                                        "QSIERRA.SIF"; "QSTAIR.SIF";
                                                        "QSTANDAT.SIF"; "UBH1.SIF"; 
                                                        "YAO.SIF"]) # problems with fixed variables
        

        println(file_name)
        pb_i = string(path_pb, "\\", file_name)
        if file_name in ["BLEND.SIF"; "DFL001.SIF"; "FORPLAN.SIF"; "GFRD-PNC.SIF"; "SIERRA.SIF";
                        "EXDATA.SIF"; "QFORPLAN.SIF"; "QGFRDXPN.SIF"; "VALUES.SIF"]
            qpdata_i = readqps(pb_i, mpsformat=:fixed) 
        else
            qpdata_i = readqps(pb_i) 
        end
        push!(problems, createQuadraticModel(qpdata_i, name=file_name[1:end-4]))
        
        if i==i_max
            break
        end
        i += 1
    end
end


25FV47.SIF
ADLITTLE.SIF
AFIRO.SIF
AGG.SIF


┌ Info: Using '25FV47' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'R0000' as objective (l. 21)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using '.00001' as RHS (l. 6793)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'ADLITTLE' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using '.Z....' as objective (l. 20)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'ZZZZ0001' as RHS (l. 333)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'AFIRO' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'COST' as objective (l. 47)
└ @ QPSReader C:\Users\Geoffroy Lecont

AGG2.SIF
AGG3.SIF
BANDM.SIF
BEACONFD.SIF
BLEND.SIF


┌ Info: Using 'OBJECTIV' as objective (l. 509)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'AGG' as RHS (l. 1852)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'AGG2' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'OBJECTIV' as objective (l. 536)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'RHS' as RHS (l. 2878)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'AGG3' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'OBJECTIV' as objective (l. 536)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'RHS' as RHS (l. 2884)
└ @ QPSReader C:\Users\Geoffroy Leconte

BNL1.SIF
BNL2.SIF


┌ Info: Using 'C' as objective (l. 98)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using '' as RHS (l. 376)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'BNL1' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'OBJ' as objective (l. 20)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'RHS' as RHS (l. 3994)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'BNL2' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'OBJ' as objective (l. 20)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'RHS' as RHS (l. 11294)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSRea

In [184]:
problems_stats = solve_problems(mehrotraPCQuadBounds, problems, 
                                colstats=[:name, :status, :elapsed_time, :objective, :dual_feas, :primal_feas])

error 11


┌ Info:            Name           status      Time      f(x)      Dual    Primal  
└ @ SolverTools C:\Users\Geoffroy Leconte\.julia\packages\SolverTools\whyGk\src\bmark\run_solver.jl:67
┌ Info:          25FV47       acceptable   8.0e-02   5.5e+03   2.4e-10   2.4e-11
└ @ SolverTools C:\Users\Geoffroy Leconte\.julia\packages\SolverTools\whyGk\src\bmark\run_solver.jl:83
┌ Info:        ADLITTLE       acceptable   0.0e+00   2.3e+05   5.1e-09   8.3e-13
└ @ SolverTools C:\Users\Geoffroy Leconte\.julia\packages\SolverTools\whyGk\src\bmark\run_solver.jl:83
┌ Info:           AFIRO       acceptable   0.0e+00  -4.6e+02   2.9e-13   4.6e-13
└ @ SolverTools C:\Users\Geoffroy Leconte\.julia\packages\SolverTools\whyGk\src\bmark\run_solver.jl:83
┌ Info:             AGG       acceptable   8.0e-03  -3.6e+07   4.7e-08   2.5e-07
└ @ SolverTools C:\Users\Geoffroy Leconte\.julia\packages\SolverTools\whyGk\src\bmark\run_solver.jl:83


error 17
error 12
error 9
error 11


┌ Info:            AGG2       acceptable   2.4e-02  -2.0e+07   3.2e-06   9.3e-09
└ @ SolverTools C:\Users\Geoffroy Leconte\.julia\packages\SolverTools\whyGk\src\bmark\run_solver.jl:83
┌ Info:            AGG3       acceptable   3.2e-02   1.0e+07   2.2e-08   1.9e-09
└ @ SolverTools C:\Users\Geoffroy Leconte\.julia\packages\SolverTools\whyGk\src\bmark\run_solver.jl:83
┌ Info:           BANDM       acceptable   8.0e-03  -1.6e+02   3.7e-10   6.3e-10
└ @ SolverTools C:\Users\Geoffroy Leconte\.julia\packages\SolverTools\whyGk\src\bmark\run_solver.jl:83
┌ Info:        BEACONFD       acceptable   1.6e-02   3.4e+04   3.0e-09   2.9e-09
└ @ SolverTools C:\Users\Geoffroy Leconte\.julia\packages\SolverTools\whyGk\src\bmark\run_solver.jl:83
┌ Info:           BLEND       acceptable   0.0e+00  -3.1e+01   1.5e-08   4.1e-11
└ @ SolverTools C:\Users\Geoffroy Leconte\.julia\packages\SolverTools\whyGk\src\bmark\run_solver.jl:83
┌ Info:            BNL1       acceptable   4.8e-02   2.0e+03   7.8e-12   3.7e-09

error 7


┌ Info:            BNL2       acceptable   3.0e-01   1.8e+03   7.9e-11   2.1e-10
└ @ SolverTools C:\Users\Geoffroy Leconte\.julia\packages\SolverTools\whyGk\src\bmark\run_solver.jl:83


11×33 DataFrame. Omitted printing of 26 columns
│ Row │ id    │ name     │ nvar  │ ncon  │ nequ  │ status     │ objective  │
│     │ Int64 │ String   │ Int64 │ Int64 │ Int64 │ Symbol     │ Float64    │
├─────┼───────┼──────────┼───────┼───────┼───────┼────────────┼────────────┤
│ 1   │ 1     │ 25FV47   │ 1571  │ 821   │ 0     │ acceptable │ 5501.85    │
│ 2   │ 2     │ ADLITTLE │ 97    │ 56    │ 0     │ acceptable │ 225495.0   │
│ 3   │ 3     │ AFIRO    │ 32    │ 27    │ 0     │ acceptable │ -464.753   │
│ 4   │ 4     │ AGG      │ 163   │ 488   │ 0     │ acceptable │ -3.59918e7 │
│ 5   │ 5     │ AGG2     │ 302   │ 516   │ 0     │ acceptable │ -2.02393e7 │
│ 6   │ 6     │ AGG3     │ 302   │ 516   │ 0     │ acceptable │ 1.03121e7  │
│ 7   │ 7     │ BANDM    │ 472   │ 305   │ 0     │ acceptable │ -158.628   │
│ 8   │ 8     │ BEACONFD │ 262   │ 173   │ 0     │ acceptable │ 33592.5    │
│ 9   │ 9     │ BLEND    │ 83    │ 74    │ 0     │ acceptable │ -30.8121   │
│ 10  │ 10    │ BNL1     │ 1175  │ 643   │ 0     │ acceptable │ 1977.63    │
│ 11  │ 11    │ BNL2     │ 3489  │ 2324  │ 0     │ acceptable │ 1811.24    │

In [ ]:
markdown_table(stdout, problems_stats)

# Résultats sur tous les problèmes netlib

In [207]:
using JLD2

In [208]:
file = jldopen("mehrotra.jld2", "w") 
file["test"] = problems_stats
close(file)


In [ ]:
using ProfileSVG

In [ ]:
pb5 = string(path_pb, "\\CRE-B.SIF")
qpdata5 = readqps(pb5)
#qpdata5 = readqps(pb5, mpsformat=:fixed)
QM5 = createQuadraticModel(qpdata5)
SM5 = SlackModel(QM5)
SM5.meta

In [ ]:
stats_mpc5 = mehrotraPCQuadBounds(SM5, max_iter=100, max_time=120., presolve=false)
println(stats_mpc5)

In [ ]:
@profview mehrotraPCQuadBounds(SM5, display=false, max_iter=50, presolve=false)

In [ ]:
ProfileSVG.save("C://Users//Geoffroy Leconte//Documents//cours//TFE//code//results//prof_geoffroy2.svg")